<a href="https://colab.research.google.com/github/katya123123/ML-labs/blob/main/ML_2_%D0%BF%D0%B5%D1%80%D0%B5%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0%D0%BD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import pandas as pd
import numpy as np
import math
from sklearn.cluster import DBSCAN

In [24]:
df = pd.read_csv("/content/student-mat.csv")
df

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,MS,M,20,U,LE3,A,2,2,services,services,...,5,5,4,4,5,4,11,9,9,9
391,MS,M,17,U,LE3,T,3,1,services,services,...,2,4,5,3,4,2,3,14,16,16
392,MS,M,21,R,GT3,T,1,1,other,other,...,5,5,3,3,3,3,3,10,8,7
393,MS,M,18,R,LE3,T,3,2,services,other,...,4,4,1,3,4,5,0,11,12,10


Категориальная переменная - sex (Y)

Числовая переменная для анализа - absences (X)

In [25]:
describe = df['absences'].describe()
print(describe)

count    395.000000
mean       5.708861
std        8.003096
min        0.000000
25%        0.000000
50%        4.000000
75%        8.000000
max       75.000000
Name: absences, dtype: float64


In [26]:
agg = df.groupby(['sex']).agg({'absences': ['mean', 'median', 'min', 'max', np.std, pd.Series.mode]})
print(agg)

     absences                              
         mean median min max       std mode
sex                                        
F    6.216346    4.0   0  75  9.445819    0
M    5.144385    4.0   0  38  5.980749    0


In [27]:
q1 = df.groupby(['sex'])['absences'].quantile(0.25)
print(q1)

sex
F    0.0
M    0.0
Name: absences, dtype: float64


In [28]:
q2 = df.groupby(['sex'])['absences'].quantile(0.5)
print(q2)

sex
F    4.0
M    4.0
Name: absences, dtype: float64


In [29]:
q3 = df.groupby(['sex'])['absences'].quantile(0.75)
print(q3)

sex
F    8.0
M    8.0
Name: absences, dtype: float64


In [30]:
iqr = q3 - q1
print(iqr)


sex
F    8.0
M    8.0
Name: absences, dtype: float64


**Выводы:**

1. Среднее в обоих группах приерно равно, различие составляет < 1
2. СРеднее отклонение отличается сильнее, чем среднее. Отклонение отличается примерно на 4
3. Однако, несмотря на то что отклонение в группе F больше, но и максимальное значение больше. Между значениями 75 и 0 среднее отклонение 9.5 примерно соотносимо со средним отклонением 5.4 между 38 и 0.
4. Медианные значения полностью совпадают, учитывая, что это более объективная переменная для анализа, чем среднее, группы имеют примерно одинаковые разбросы данных.
5. Квантили во обоих группах абсолютно одинаковые, что подтверждает предыдущее предположение.
6. Учитывая, что медиана и квантили в обоих группах одинаковые, но максимальные значения очень сильно отличаются, можно сделать вывод, что в первой группе есть сильный (или сильные) выбросы.
7. Таким образом, среднее в группе F больше, поскольку на него оказывает влияние выброс (выбросы)


In [31]:
import plotly.express as plt
%matplotlib inline

In [32]:
fig = plt.histogram(x = df['absences'], y = df['sex'])
fig.show()

In [33]:
fig = plt.box(df, y='absences')
fig.show()

In [34]:
fig = plt.box(df, x = 'sex', y = 'absences')
fig.show()

In [35]:
fig = plt.scatter(df, x = 'age', y = 'absences')
fig.show()

In [167]:
corr_matrix = df.corr()
fig = plt.imshow(corr_matrix)
fig.show()

#Примечание: мне показалось, что по двум числовым
#переменным - это совершенно не показательно и ни о чем не скажет, поэтому взяла хотя бы три

<ipython-input-167-a33e88220890>:1: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



Судя по полученной матрице корреляции, критерии крайне мало влияют друг на друга. Соответственно, в данном датасете очень низкий уровень связности. Взаимозависимыми являются только показатель G1, G2, G3, а также можно видеть зависимость между количеством выходов с друзьями и количеством пьянок на выходных и в будние дни.

Касательно остальных графиков, можно видеть, что количество прогулов имеет выбросы, при чем выбросы в группе среди женщин больше. Соотвественно,в школе есть девочки, которые прогуливали очень очень много.

По матрице рассеивания можно сделать вывод, что пиком количества прогулов является возраст с 16 до 19, с максимальным пиком в 17 лет.

**Задание для гениев**

In [37]:
mean = df['absences'].mean()
median = df['absences'].median()
std = df['absences'].std()
N = df['absences'].count()
print("mean: ", mean, "median: ", median)
print("Выборка: ")
if median - mean <= 3 * math.sqrt(std**2/N):
  print("симметричная")
  cof = 3
else:
  print("несимметричная")
  cof = 5

mean:  5.708860759493671 median:  4.0
Выборка: 
симметричная


In [163]:
class Outline:
  def __init__(self, df):
    self.df = df

  def del_outline_by_mean(self, col):
    arr =[]
    min = mean - cof * std
    max = mean + cof * std
    for ind, v in self.df[col].items():
      if (v <= min) or (v >= max):
        arr.append(ind)

    return print("Индексы выбросов по среднему", arr)

  def del_outline_by_shovene(self, col):
    df = self.df[col].copy()
    arrSh = []
    n = df.count()
    for  ind, v in df.items():
      meanSh = df.mean()
      stdSh = df.std()
      if math.erfc((abs(v-meanSh))/stdSh) < 1/(2*n):
        arrSh.append(ind)
      else:
        n -= 1
        df = df.drop(index = ind)

    return print("Индексы выбросов по Шовене: ", arrSh)

  def del_outline_by_qu(self, col):
    arrQ = []
    Q1 = self.df[col].quantile(0.25)
    Q2 = self.df[col].quantile(0.5)
    Q3 = self.df[col].quantile(0.75)
    min = Q1-1.5*(Q3-Q1)
    max = Q3+1.5*(Q3-Q1)
    for ind, v in self.df[col].items():
      if (v <= min) or (v >= max):
        arrQ.append(ind)
    return print("Индексы выбросов по квартилям: ", arrQ)

  def del_outline_by_DBSCAN(self, col1, col2):
    new_df = self.df[[col1, col2]]
    temp_df = self.df[[col1, col2]]
    cluster = DBSCAN(eps = 2, min_samples = 1).fit(new_df)
    new_df['labels']  = cluster.labels_
    temp_df['labels']  = cluster.labels_
    labels= list(new_df['labels'])
    uniq, count = np.unique(labels, return_counts=True)
    dict_of_uniq= dict(zip(uniq, count))
    outline_cluster = []
    for k,v in dict_of_uniq.items():
      if v <=2:
        outline_cluster.append(k)
    temp_df = new_df[new_df['labels'].isin(outline_cluster)].index
    new_df = new_df[~new_df['labels'].isin(outline_cluster)]
    fig = plt.scatter(new_df, x = col1, y = col2, color = 'labels')
    fig.show()
    print(temp_df)


In [164]:
ind_of_outline = Outline(df)
ind_of_outline.del_outline_by_mean("absences")
ind_of_outline.del_outline_by_shovene("absences")
ind_of_outline.del_outline_by_qu("absences")


Индексы выбросов по среднему [74, 183, 276, 280, 307, 315]
Индексы выбросов по Шовене:  [40, 74, 103, 183, 276, 307, 315, 393]
Индексы выбросов по квартилям:  [40, 74, 103, 118, 183, 198, 205, 216, 237, 260, 276, 277, 280, 304, 307, 311, 313, 315, 320]


In [165]:
ind_of_outline.del_outline_by_DBSCAN("absences", "age")

<ipython-input-163-0c3682b8c45f>:46: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-163-0c3682b8c45f>:47: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Int64Index([74, 183, 205, 247, 276, 280, 307, 315], dtype='int64')
